<a href="https://colab.research.google.com/github/denisich2001/KonturIntership/blob/main/Kontur_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Считываем и очищаем тестовые данные

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
trainData = pd.read_table("train.tsv")
trainData

,title,is_fake
0,Москвичу Владимиру Клутину пришёл счёт за вмеш...,1
1,Агент Кокорина назвал езду по встречке житейск...,0
2,Госдума рассмотрит возможность введения секрет...,1
3,ФАС заблокировала поставку скоростных трамваев...,0
4,Против Навального завели дело о недоносительст...,1
...,...,...
5753,Эдди Чемберс получил сотрясение мозга в бою с ...,0
5754,Правительство застроит Россию нефтепродуктопро...,0
5755,«Мне стыдно перед дедом»: новый канцлер ФРГ об...,1
5756,Туркмения в декабре начнет поставки газа в Китай,0


In [ ]:
from collections import Counter
print(Counter(trainData['is_fake'].values))

Counter({1: 2879, 0: 2879})


In [ ]:
trainData.isnull().sum()

title      0
is_fake    0
dtype: int64

Отделяем данные от значений

In [ ]:
trainDataX = trainData.drop(columns = 'is_fake', axis = 1)
trainDataX = trainDataX['title'].values

trainDataY = trainData['is_fake']

#Очищаем и предобрабатываем данные

In [ ]:
!pip install pymorphy2
nltk.download("russian")

[nltk_data] Error loading russian: Package 'russian' not found in
[nltk_data]     index


False

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
import regex as re
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Разбиваем на токены, удаляем стоп-слова и лемматизируем

In [ ]:
def preprocessing(data):
  #Токенизируем, удаляем стоп-слова
  wordsList = re.sub('[^а-яА-ЯёЁ]',' ',data)
  wordsList = wordsList.split()
  wordsList = [word.lower() for word in wordsList if not word in stopwords.words('russian')]
  
  #Лемматизируем
  wordsList = [morph.normal_forms(word)[0] for word in wordsList]

  wordsList = ' '.join(wordsList)
  return wordsList

#Предобрабатываем тренировочные данные
for index, line in enumerate(trainDataX):
  trainDataX[index] = preprocessing(line)


In [ ]:
trainDataX

array(['москвич владимир клутина прийти счёт вмешательство американский выборы',
       'агент кокорин назвать езда встречка житейский история',
       'госдума рассмотреть возможность введение секретный стать уголовный кодекс',
       ...,
       'я стыдно дед новый канцлер фрг обратиться первый речь бундестаг',
       'туркмения декабрь начать поставка газ китай',
       'бывший тренер локомотив возглавить нальчикский спартак'],
      dtype=object)

**Векторизуем данные**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=2)
vectorizer.fit(trainDataX)

#векторизуем тренировочные данные
trainDataX1 = vectorizer.transform(trainDataX)

#Обучаем модели на обработанных данных и измеряем их точность

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

trainX, testX, trainY, testY = train_test_split(trainDataX1, trainDataY, stratify = trainDataY, random_state = 2, test_size = 0.25)

In [ ]:
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression()
model1 .fit(trainX, trainY)

predY1 = model1.predict(testX)

modelAccuracy = f1_score(predY1, testY)
modelAccuracy

0.8367952522255193

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
model2 = PassiveAggressiveClassifier(max_iter=1000)

model2.fit(trainX, trainY)
predY2 = model2.predict(testX)

modelAccuracy = f1_score(predY2, testY)
modelAccuracy

0.8115107913669066

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model3 = MultinomialNB()
model3.fit(trainX, trainY)
predY3= model3.predict(testX)

modelAccuracy = f1_score(predY3, testY)
modelAccuracy

0.8472222222222222

In [ ]:
from xgboost import XGBClassifier

model4 = XGBClassifier()
model4.fit(trainX, trainY)
predY4 = model4.predict(testX)

modelAccuracy = f1_score(predY4, testY)
modelAccuracy

0.5848708487084872

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model5 = RandomForestClassifier(n_estimators=23)
model5.fit(trainX, trainY)
predY5 = model5.predict(testX)

modelAccuracy = f1_score(predY5, testY)
modelAccuracy

0.7607655502392345

**Заключение**: проверив 5 разных моделей, можно сделать вывод, что наиболее точным был Байесовский классификатор. Его и используем для классификации тестовых данных

#Размечаем тестовые данные

In [ ]:
testData = pd.read_table("predictions.tsv")
testData = testData['title'].values

In [ ]:
#Предобрабатываем тестовые данные
for index, line in enumerate(testData):
  testData[index] = preprocessing(line)

testData[:10]

array(['роскомнадзор представить реестр сочетание цвет нежелательный россия',
       'ночью минский президентский гора беларашмор пик демократия внезапно появиться лицо николай лукашенко',
       'бывший спичрайтер юрий лоза рассказать трудность работа сергей шойгу',
       'сельский церковь собрать рекордно низкий количество пожертвование сгореть тройной попадание молния',
       'акция рухнуть объявление перезапуск',
       'курс доллар вырасти исторический максимум',
       'опек назвать оптимальный уровень цена нефть',
       'российский авиакомпания открыть рейс тбилиси урал',
       'швейцарский горнолыжница расстрелять дом родитель',
       'учредить театральный премия имя гарольд пинтер'], dtype=object)

In [ ]:
#векторизуем тестовые данные
testData1 = vectorizer.transform(testData)

In [ ]:
testDataPredictions = model3.predict(testData1)
#testDataPredictions

In [ ]:
result = pd.read_table("predictions.tsv")
result['is_fake'] = testDataPredictions
result[:10]

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,1
1,Ночью под Минском на президентской горе Белара...,1
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,1
3,"Сельская церковь, собравшая рекордно низкое ко...",1
4,Акции Google рухнули после объявления о переза...,0
5,Курс доллара вырос до исторического максимума,0
6,ОПЕК назвала оптимальный уровень цен на нефть,0
7,Российская авиакомпания откроет рейсы в Тбилис...,0
8,Швейцарская горнолыжница расстреляна в доме ро...,1
9,Учреждена театральная премия имени Гарольда Пи...,0


In [ ]:
result.to_csv('predictions.tsv', sep="\t", index=False)